# 第六組第四次測試
測試照片原圖路徑: ./yolov7-main/Final/imagesTest
測試照片答案路徑: ./yolov7-main/Final/imageAnswer
預測照片答案路徑: ./yolov7-main/Final/imagePredict
預測照片標籤路徑: ./yolov7-main/runs/exp/labels

# 環境
torch>=1.7.0,!=1.12.0
torchvision>=0.8.1,!=0.13.0
tqdm>=4.41.0
tensorboard>=2.4.1

### import model

In [1]:
import cv2
import os
import glob
import numpy as np
import pandas as pd
import json

### AnswerMaker use txt to image

In [2]:
def txt_to_image(txt_path, image_path, save_image_path):
    txt = os.listdir(txt_path)
    image = os.listdir(image_path)

    # 設定圖片大小
    width = 640
    high = 360

    for idx_img,imageName in enumerate(image):
        imageOut=np.zeros((high,width,3))
        file = open(txt_path+txt[idx_img], 'r')
        recs=file.readlines()
        for rec in recs:
            info=rec[:-1].split(' ')
            cx=int(float(info[1])*640)
            cy=int(float(info[2])*360)
            w=int(float(info[3])*640)
            h=int(float(info[4])*360)
            x1=cx-w//2
            y1=cy-h//2
            x2=cx+w//2
            y2=cy+h//2

            if int(info[0])==0:
                imageOut=cv2.rectangle(imageOut, (x1, y1), (x2, y2), (0, 0, 255), -1)
            else:
                imageOut=cv2.rectangle(imageOut, (x1, y1), (x2, y2), (0, 255, 0), -1)
        
        cv2.imwrite(save_image_path+imageName, imageOut)

### Predict images

In [3]:
def Preprocess(test_data_path):
    # Input image pre-processing
    # Process the indput image to conform to the shape and format of the model
    # return test_data
    pass

def LoadModel(pretrained_model_file):
    # Load deep learning model
    # return model
    pass

def Predict():
    # return prediction
    # use yolov7 detect.py
    !python detect.py --save-txt --weights ./weights/best.pt --conf 0.25 --img-size 640 --source ./Final/imagesTest
    return


def Postprocess(prediction):
    # Post-precessing
    # Save the prediction results as an image and use the filename "prediction_n"
    # return prediction_path
    pass

### Score calculater

In [4]:
import os
import cv2
import numpy as np
import glob
import pandas as pd
from keras import backend as K
import  tensorflow

def dice_coef(y_true, y_pred, smooth=0):
    intersection = K.sum(y_true * y_pred, axis=[0,1])
    union = K.sum(y_true, axis=[0,1]) + K.sum(y_pred, axis=[0,1])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def Calculate_Score(answer_path, prediction_path):
    answer_path = answer_path
    prediction_path = prediction_path
    answer_list = os.listdir(answer_path)
    prediction_list = os.listdir(prediction_path)
    #print(answer_list)
    #print(prediction_list)
    
    B_score = []
    G_score = []
    R_score = []
    for i in range(len(answer_list)):
        ans = cv2.imread(f'{answer_path}{answer_list[i]}').astype('float32')
        ans[:, :, 0] = np.where(ans[:, :, 0] > 0, 0, 0)
        ans[:, :, 1] = np.where(ans[:, :, 1] > 1, 1, 0)
        ans[:, :, 2] = np.where(ans[:, :, 2] > 1, 1, 0)
        pre = cv2.imread(f'{prediction_path}{prediction_list[i]}').astype('float32')
        pre[:, :, 0] = np.where(pre[:, :, 0] > 0, 0, 0)
        pre[:, :, 1] = np.where(pre[:, :, 1] > 1, 1, 0)
        pre[:, :, 2] = np.where(pre[:, :, 2] > 1, 1, 0)

        
        for j in range(1,3):
            # BGR
            dice = dice_coef(ans[:,:,0+j:1+j], pre[:,:,0+j:1+j])
            dice_score = dice.numpy()
            if np.isnan(dice_score):
                print(prediction_list[i])
            
            if j == 0:
                B_score.append(dice_score)
            elif j == 1:
                G_score.append(dice_score)
            else:
                R_score.append(dice_score)
    print(np.mean(B_score))
    print(np.mean(G_score))
    print(np.mean(R_score))
    print('The final score : ', np.mean(G_score)+np.mean(R_score))


### main function

In [5]:
Predict()

# generate predict images
txt_to_image('./runs/detect/exp/labels/','./Final/imagesTest/','./Final/imagePredict/')

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=True, source='./Final/imagesTest', update=False, view_img=False, weights=['./weights/best.pt'])
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 0, 2 1s, Done. (157.7ms) Inference, (46.8ms) NMS
 The image with the result is saved in: runs\detect\exp\105.jpg
1 0, 1 1, Done. (156.2ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\1080.jpg
1 0, 2 1s, Done. (161.8ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\1101.jpg
1 0, 2 1s, Done. (156.2ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\116.jpg
2 0s, 3 1s, Done. (162.4ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp

YOLOR  2023-1-1 torch 1.8.2+cu111 CUDA:0 (NVIDIA GeForce GTX 1050, 4095.8125MB)

Model Summary: 362 layers, 70789182 parameters, 0 gradients, 188.0 GFLOPS


In [6]:
Calculate_Score('./Final/imageAnswer/', './Final/imagePredict/')

nan
0.7193978
0.6896347
The final score :  1.4090325


C:\Users\justi\anaconda3\envs\yolov7\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\justi\anaconda3\envs\yolov7\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
